# **Logbook es. 10**

## **Introduzione**

Con la seguente esperienza si vogliono esplorare capacità e funzionamento del sensore inerziale fornitoci per la pausa invernale.

### **Task 1**

Obiettivo: Scegliere due canali digitali a piacere fra i 16 disponibili con nomi che vanno da DIO0 (digital input/output numero 0) a DIO15 (digital input/output numero 15), in seguito i canali scelti verranno indicati con A e B. Dopodichè eseguire le sequenti azioni:

* Selezionare la modalità Patterns (generatore di segnali digitali) e aggiungere un singolo segnale su A, selezionare output DD, tipo Random, una frequenza di 500Hz e avviare la generazione con il tasto RUN;

* Cortocircuitare A con uno degli ingressi analogici di Analog Discovery 2, per esempio Ch1 (ricordare la terra!), e verificare la corretta generazione di un segnale di tipo digitale;

* Selezionare la modalità Logic (oscilloscopio digitale) e definire un bus a cui aggiungere il secondo canale digitale scelto B: scollegare A dall’ingresso analogico e collegarlo a B per verificare la corretta misura
ed identificazione dei livelli digitali.

Si sono scelte le porte digitali DIO1 e DIO2, rispettivamente A e B.

Prima si è impostato il canale A, ottenendo il seguente risultato:

<img src="DIO1_task1.png" alt="impostazione DIO1" width="800" height="50"/>

*Figura 1: risultato ottenuto impostando DIO1 come richiesto.*

Poi si osservata l'uscita della porta all'oscilloscopio:

<img src="osc_DIO1_task1.png" alt="oscilloscopio DIO1" width="800" />

*Figura 2: osservazione all'oscilloscopio di DIO1.*

Dopo di che si è anche impostato il canale DIO2 come bus, impostando anche il clock per questo canale: DIO1.

<img src="bus_task1.png" alt="bus DIO2" width="1000"  height="100"/>

*Figura 3: osservazione all'oscilloscopio di DIO1.*

### **Task 2 – Parametri dal datasheet del MPU6050**

**Obiettivo:**
- Estrarre dal datasheet, per le grandezze misurate (accelerazione, velocità angolare, temperatura*):
  - **range dinamico**,
  - **sensibilità** (LSB - least significant bit - per unità fisica),
  - **accuratezza** (tolleranze di calibrazione, offset, ecc.),
  - **precisione** (rumore, ripetibilità, dipendenze da configurazione - velocità di acquisizione, numero di medie fatte dal chip stesso).

*il sensore integra un sensore di temperatura interno che misura la temperatura del chip, non la temperatura ambientale, utile per interpretare eventuali drift termici delle misure di accelerazione e velocità angolare.


**GIROSCOPIO** (specifiche di voltaggio):
| Parametro                     | Valore                         |
|-------------------------------|--------------------------------|
| $V_{DD}$    (T=25°C)          | 2.375 V – 3.46 V               |
| $V_{LOGIC}$ (T=25°C)          | 1.8 V $\pm$ 5%                 |

Range dinamico (full - scale):

<img src="dinamic_range_gyroscope.png" alt="specifiche giroscopio" width="800"/>

*Figura 4: range dinamico del giroscopio; le misure vengono salvate in due byte (Word Lenght = 16 bits)*

Sensibilità (Scale Factor):

<img src="sensitivity_gyroscope.png" alt="sensibilità giroscopio" width="800"/>

*Figura 5: sensibilità del giroscopio in funzione del fondoscala selezionato.*

Ad esempio, 1 LSB = 0.00763 °/s per un fondoscala di ±250 °/s. Cioè, per ogni 1 °/s di velocità angolare reale,
il registro digitale cambia di 131 unità numeriche (LSB).

Accuratezza:

<img src="tolerance_gyroscope.png" alt="accuratezza giroscopio" width="800"/>

*Figura 6: accuratezza del giroscopio in vari parametri.*

N.B.: 
- la cross-axis sensitivity indica la sensibilità del giroscopio alle accelerazioni angolari applicate lungo gli assi ortogonali a quello di misura;
- ZRO (Zero Rate Output) indica l'offset del giroscopio a riposo, cioè la lettura del sensore quando non viene applicata alcuna velocità angolare.

Precisione:

<img src="precision_gyroscope.png" alt="precisione giroscopio" width="800">

*Figura 7: precisione del giroscopio in vari parametri.*

In pratica: la precisione migliora filtrando e/o mediando più campioni, ma si perde banda.

**Dipendenza dalla configurazione**
- **FS_SEL / AFS_SEL**: determinano il range dinamico e il fattore di scala (sensibilità).
- **DLPF_CFG**: determina la banda passante effettiva e il livello di rumore.
- **Sample Rate / ODR (output data rate)**: interagisce con il DLPF e definisce il numero di campioni interni effettivamente “mediati” nel tempo.

<img src="configuration_dependancy.png" alt="configurazione giroscopio" width="800">

*Figura 8: dipendenza dalla configurazione.*

**ACCELEROMETRO** (specifiche di voltaggio):
| Parametro                     | Valore                         |
|-------------------------------|--------------------------------|
| $V_{DD}$    (T=25°C)          | 2.375 V – 3.46 V               |
| $V_{LOGIC}$ (T=25°C)          | 1.8 V $\pm$ 5%                 |  

Range dinamico (full - scale):

<img src="dinamic_range_accelerometer.png" alt="specifiche accelerometro">

*Figura 9: range dinamico dell'accelerometro; le misure vengono salvate in due byte (Word Lenght = 16 bits)*

Sensibilità (Scale Factor):

<img src="sensitivity_accelerometer.png" alt="sensibilità accelerometro" width="800"/>

*Figura 10: sensibilità dell'accelerometro in funzione del fondoscala selezionato.*

Ad esempio, 1 LSB = 0.000061 g per un fondoscala di ±2 g. Cioè, per ogni 1 g di accelerazione reale,
il registro digitale cambia di 16384 unità numeriche (LSB).

Accuratezza:

<img src="tolerance_accelerometer.png" alt="accuratezza accelerometro" width="800"/>

*Figura 11: accuratezza dell'accelerometro in vari parametri.*

Precisione:

<img src="precision_accelerometer.png" alt="precisione accelerometro" width="800">

*Figura 12: precisione dell'accelerometro.*

Con dipendenza dalla configurazione simile a quella del giroscopio.

**TEMPERATURA**

<img src="temperature_parameters.png" alt="specifiche temperatura" width="800"/>

*Figura 13: parametri del sensore di temperatura integrato.*











### **Task 3 – Montaggio del sensore e verifiche di base**

**Obiettivo:**
- Comprendere il significato dei pin del MPU6050;
- Verificare che l’alimentazione disponibile (3.3 V di AD2) sia compatibile con il sensore;
- Collegare correttamente i pin I²C (SCL, SDA) ad AD2 e determinare sperimentalmente l’indirizzo I²C (SAD) tramite uno script di scansione.

### Significato dei pin del sensore (con riferimento al datasheet)

Secondo il datasheet, sezione 7.1 “Pin Out and Signal Description” (pag. 21), i pin principali del sensore hanno le seguenti funzioni:

#### Pin di alimentazione
- **VDD**  
  Alimentazione principale del chip.  
  Il datasheet specifica un range operativo 2.375 V – 3.46 V (pag. 6).  
  L’alimentazione disponibile a 3.3 V risulta quindi compatibile.
- **GND**  
  Riferimento di massa del dispositivo.

#### Pin dell’interfaccia I²C primaria
- **SCL**  
  Serial Clock Line del bus I²C.
- **SDA**  
  Serial Data Line del bus I²C.  

#### Pin di selezione indirizzo
- **AD0**  
  Seleziona il bit meno significativo dell’indirizzo I²C dello slave.  
  Dal datasheet (pag. 33):
  - AD0 = 0 → indirizzo **0x68**
  - AD0 = 1 → indirizzo **0x69**


### Altri pin del sensore 

#### **INT**
- Pin di **interrupt digitale** (uscita).
- Può segnalare eventi come:
  - Data Ready
  - Overflow FIFO
  - Motion detection
- Il comportamento è configurabile tramite i registri `INT_ENABLE` e `INT_PIN_CFG` (sezione 8, pag. 27–29).
- **Non è necessario** per una comunicazione I²C di base, ma è utile per acquisizioni sincronizzate o a basso consumo.

#### **XDA** e **XCL**
- Appartengono all’**Auxiliary I²C Bus** del MPU-6050.
- **XDA**: linea dati (Aux SDA)  
- **XCL**: linea clock (Aux SCL)
- Questo bus è utilizzato per collegare **sensori esterni** (es. magnetometro) che vengono poi letti internamente dal MPU e resi disponibili nei suoi registri (sezione **Auxiliary I²C Interface**, pag. 30–32).
- In questa esperienza **non vengono utilizzati** e possono essere lasciati non connessi.


### Verifica dell’indirizzo I²C (SAD)

Il datasheet indica che l’indirizzo I²C del dispositivo è della forma:

$\text{b110100X}$ (b: binario)

dove **X è il livello logico del pin AD0**.  
Nel montaggio utilizzato, AD0 risulta basso (connesso a GND), quindi l’indirizzo atteso è **0x68** (0x: esadecimale).

<img src="collegamenti_pin.jpg" alt="verifica indirizzo I2C" width="400"/>

*Figura 14: collegamenti dei pin del sensore MPU6050 ad AD2.*




In [1]:
import tdwf

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()
#   2. Configurazione alimentazione
ad2.vdd = 3.3
ad2.power(True)
#   3. Impostazione bus I2C
i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
devs = i2c.scan()  # verifica cosa è connesso...
for dev in devs:
    print(f"Found device @SAD: 0x{dev:02x}")

# ---------------------------------------

ad2.close()

Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Found device @SAD: 0x68
Dispositivo disconnesso.


Lo script di scansione del bus I²C ha individuato un dispositivo all’indirizzo **0x68**, confermando la coerenza tra:
- collegamento del pin AD0
- indirizzo riportato dal datasheet
- indirizzo rilevato sperimentalmente

### **Task 4 – Comandi, registri e conversione in unità fisiche**

**Obiettivo:**
- Individuare i **registri principali** del MPU6050:
  - registri di configurazione (fondoscala, filtri, frequenze di campionamento),
  - registri di gestione (es. PWR_MGMT_1),
  - registri dati per accelerometro, giroscopio e temperatura.
- Annotare le **formule** per convertire i dati grezzi (LSB) in unità fisiche (g, °/s, °C).
- Interrogare una AI generativa con un prompt strutturato per farsi aiutare a costruire una tabella dei registri e confrontare criticamente le risposte con il datasheet.

### Registri di gestione e configurazione

- **`PWR_MGMT_1` (0x6B)**  
  Registro di gestione dell’alimentazione.  
  Permette di:
  - resettare il dispositivo (`DEVICE_RESET`)
  - uscire dalla modalità sleep
  - selezionare la sorgente di clock  
  È il primo registro da configurare dopo l’accensione.

- **`PWR_MGMT_2` (0x6C)**  
  Gestione dell’abilitazione dei singoli sensori.  
  Consente di disabilitare o abilitare accelerometro e giroscopio sui vari assi.

- **`WHO_AM_I` (0x75)**  
  Registro di identificazione del dispositivo.  
  Restituisce il valore fisso **0x68**, utile per verificare la corretta comunicazione I²C.

- **`SMPLRT_DIV` (0x19)**  
  Divisore del sample rate.  
  La frequenza di campionamento finale è:
  $$
  f_{sample} = \frac{f_{gyro}}{1 + \text{SMPLRT\_DIV}}
  $$

- **`CONFIG` (0x1A)**  
  Registro di configurazione generale.  
  Contiene i bit per impostare il **Digital Low Pass Filter (DLPF)**, che influenza banda passante e frequenza di campionamento interna.

- **`GYRO_CONFIG` (0x1B)**  
  Configura il giroscopio:
  - selezione del full-scale
  - eventuali opzioni avanzate di test.

- **`ACCEL_CONFIG` (0x1C)**  
  Configura l’accelerometro:
  - selezione del full-scale,
  - impostazioni di test.

### Registri di uscita: temperatura

- **`TEMP_OUT_H` (0x41)**  
- **`TEMP_OUT_L` (0x42)**  

Contengono il valore grezzo (**16 bit, complemento a due**) della temperatura interna del sensore.
La temperatura in gradi Celsius si calcola come:
$$
Temp [°C] = \frac{TEMP\_OUT}{340} + 36.53
$$

### Registri di uscita: accelerazione (3 assi)

- **`ACCEL_XOUT_H` (0x3B)**  
- **`ACCEL_XOUT_L` (0x3C)**  
- **`ACCEL_YOUT_H` (0x3D)**  
- **`ACCEL_YOUT_L` (0x3E)**  
- **`ACCEL_ZOUT_H` (0x3F)**  
- **`ACCEL_ZOUT_L` (0x40)**  

Ogni coppia di registri rappresenta un asse dell’accelerometro (X, Y, Z) come **valore a 16 bit in complemento a due**.
La conversione in unità fisiche (g) dipende dal full-scale selezionato (come rappresentato nella tabella di sensibilità):
$$
Accel [g] = \frac{ACCEL\_OUT}{Sensitivity\_Factor}
$$

### Registri di uscita: velocità angolare (giroscopio, 3 assi)

- **`GYRO_XOUT_H` (0x43)**  
- **`GYRO_XOUT_L` (0x44)**  
- **`GYRO_YOUT_H` (0x45)**  
- **`GYRO_YOUT_L` (0x46)**  
- **`GYRO_ZOUT_H` (0x47)**  
- **`GYRO_ZOUT_L` (0x48)**  

Ogni coppia di registri rappresenta un asse del giroscopio (X, Y, Z) come **valore a 16 bit in complemento a due**.
La conversione in unità fisiche (°/s) dipende dal full-scale selezionato (come rappresentato nella tabella di sensibilità):
$$
Gyro [°/s] = \frac{GYRO\_OUT}{Sensitivity\_Factor}
$$


### Osservazione

Questi registri costituiscono il set minimo per:
- inizializzare correttamente il sensore,
- verificare la comunicazione I²C,
- acquisire le misure di **temperatura**, **accelerazione** e **velocità angolare** lungo i tre assi.

Essi sono quelli effettivamente da noi utilizzati nel corso dell'esperienza.

Di seguito, riportiamo la risposta fornita da due AI (ChatGPT e Gemini) con il prompt suggerito nella scheda:

**CHATGPT**:

<img src="chatgpt1.png" alt="risposta chatgpt" width="400"/>
<img src="chatgpt2.png" alt="risposta chatgpt" width="400"/>
<img src="chatgpt3.png" alt="risposta chatgpt" width="400"/>
<img src="chatgpt4.png" alt="risposta chatgpt" width="400"/>
<img src="chatgpt5.png" alt="risposta chatgpt" width="400"/>

*Figura 15: risposta di ChatGPT al prompt strutturato.*

**GEMINI**:

<img src="gemini1.png" alt="risposta gemini" width="400"/>
<img src="gemini2.png" alt="risposta gemini" width="400"/>
<img src="gemini3.png" alt="risposta gemini" width="400"/>

*Figura 16: risposta di Gemini al prompt strutturato.*

Confrontando le risposte delle AI con lo studio standard del datasheet e del register map riportato si nota che:
- Entrambe le AI forniscono una panoramica abbastanza completa dei registri principali del MPU6050, includendo sia i registri di configurazione che quelli di output; tuttavia, ChatGPT fornisce un insieme più completo dei registri utili, ma in entrambi i casi mancano dei registri fondamentali (come il PWR_MGMT_2) e, nel caso di GPT, ci sono dei registri che non sono effettivamente indispensabili all'utilizzo che ne facciamo noi, cioè l'INT_ENABLE e l'INT_STATUS. Inoltre, Gemini dimentica il GYRO_OUT, che invece è assolutamente necessario.
- In entrambi i casi mancano alcuni riferimenti alle pagine, e anche laddove presenti spesso sono sbagliate.
- Le formule per la conversione della temperatura è sbagliato in Gemini (la costante additiva è 36.53, non 35); in ChatGPT sono tutte e tre (accelerazione, velocità angolare, temperatura) corrette, ma manca la spiegazione del fatto che i valori grezzi sono in complemento a due.
- I procedimenti di misura descritti da entrambe le AI sono corretti, ma nel nostro caso abbiamo preferito separare e visualizzare le misure di temperatura, accelerazione e velocità angolare in tre blocchi distinti, per chiarezza.

Come miglioria, si potrebbe chiedere alle AI di fornire riferimenti più precisi alle pagine del datasheet e di includere spiegazioni più dettagliate sulle modalità di lettura dei registri (ad esempio, come combinare i byte alti e bassi per ottenere i valori a 16 bit).



### **Task 5 – Script di lettura periodica (I2C_MPU6050_template)**

**Obiettivo:**
- Modificare lo script `I2C_MPU6050_template` per:
  - leggere i registri di misura,
  - convertire i dati grezzi in unità fisiche,
  - riporti il risultato sulla connessione seriale.

**Descrizione:**
- Si parte dal notebook/template che già comunica con il sensore e legge almeno la temperatura.
- Si aggiungono:
  - la scrittura nel registro `PWR_MGMT_1` per svegliare il sensore,
  - la configurazione dei fondoscala di accelerometro e giroscopio (scrittura nei registri `ACCEL_CONFIG` e `GYRO_CONFIG`),
  - funzioni di utilità per leggere parole a 16 bit (high + low) e convertirle in interi con segno (two’s complement),
  - la conversione in unità fisiche usando i fattori di scala del Task 2.
- Si implementa un ciclo principale che:
  - legge le misure sugli assi $(a_x, a_y, a_z)$, $(\omega_x, \omega_y, \omega_z)$ e la temperatura,
  - stampa i valori in formato leggibile e/o li scrive su file (es. CSV) per l’analisi statistica nel Task 6.





In [3]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import time
import math

# -[MPU6050 - Registri principali]---------------------------------------------
PWR_MGMT1    = 0x6B
PWR_MGMT2   = 0x6C
WHO_AM_I     = 0x75
SMPLRT_DIV   = 0x19
CONFIG       = 0x1A
GYRO_CONFIG  = 0x1B
ACCEL_CONFIG = 0x1C

# Blocco misure: burst-read da 0x3B per 14 bytes
ACCEL_XOUT_H = [0x3B]      # start register (must be list for tdwf.writeread)
SAD = 0x68

# Scale factors (default full-scale)
ACC_SCALE = 16384.0        # LSB/g  for ±2g (AFS_SEL=0)
GYR_SCALE = 131.0          # LSB/(deg/s) for ±250 dps (FS_SEL=0)

# -[Utility]-------------------------------------------------------------------
def twos_comp_16(val):
    """Convert unsigned 16-bit to signed 16-bit (two's complement)."""
    if val & 0x8000:
        val -= 0x10000
    return val

def parse_burst14(vals):
    """
    vals: list of 14 bytes from 0x3B..0x48
    returns: ax[g], ay[g], az[g], T[°C], gx[°/s], gy[°/s], gz[°/s]
    """
    # accel
    ax_raw = twos_comp_16((vals[0] << 8) | vals[1])
    ay_raw = twos_comp_16((vals[2] << 8) | vals[3])
    az_raw = twos_comp_16((vals[4] << 8) | vals[5])
    # temp
    t_raw  = twos_comp_16((vals[6] << 8) | vals[7])
    # gyro
    gx_raw = twos_comp_16((vals[8]  << 8) | vals[9])
    gy_raw = twos_comp_16((vals[10] << 8) | vals[11])
    gz_raw = twos_comp_16((vals[12] << 8) | vals[13])

    ax = ax_raw / ACC_SCALE
    ay = ay_raw / ACC_SCALE
    az = az_raw / ACC_SCALE

    gx = gx_raw / GYR_SCALE
    gy = gy_raw / GYR_SCALE
    gz = gz_raw / GYR_SCALE

    T  = t_raw / 340.0 + 36.53

    return ax, ay, az, T, gx, gy, gz

# -[Configurazione AD2]--------------------------------------------------------
ad2 = tdwf.AD2()
try:
    ad2.vdd = 3.3
    ad2.power(True)

    # Bus I2C
    i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
    devs = i2c.scan()
    for dev in devs:
        print(f"Device: 0x{dev:02x}")

    # Device I2C
    sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

    # -[Configurazione sensore]-------------------------------------------------
    # Reset (robusto): scrivi reset e attendi che il bit 7 torni 0
    sht.write([PWR_MGMT1, 0x80])
    time.sleep(0.01)
    t0 = time.time()
    while True:
        time.sleep(0.01)
        sht.writeread([PWR_MGMT1], 1)
        if (sht.vals[0] & 0x80) == 0:
            break
        if time.time() - t0 > 1.0:
            print("WARNING: reset timeout")
            break

    # Wake + enable sensors
    sht.write([PWR_MGMT1, 0x00])
    time.sleep(0.05)
    sht.write([PWR_MGMT2, 0x00])
    time.sleep(0.01)

    # Config filtro e sample rate (come nel tuo codice)
    sht.write([CONFIG, 0x03])
    time.sleep(0.01)
    sht.write([SMPLRT_DIV, 0x04])
    time.sleep(0.01)

    # Full-scale (default: accel ±2g, gyro ±250 dps)
    sht.write([GYRO_CONFIG, 0x00])
    time.sleep(0.01)
    sht.write([ACCEL_CONFIG, 0x00])
    time.sleep(0.01)

    # Check WHO_AM_I
    sht.writeread([WHO_AM_I], 1)
    print(f"WHO_AM_I: 0x{sht.vals[0]:02x}")

    # -[Gestione eventi]--------------------------------------------------------
    def on_close(event):
        global flag_run
        flag_run = False

    def on_key(event):
        global flag_run
        if event.key == 'escape':
            flag_run = False

    # -[Plot live]--------------------------------
    fig, ax = plt.subplots(figsize=(12,6))
    fig.canvas.mpl_connect("close_event", on_close)
    fig.canvas.mpl_connect("key_press_event", on_key)

    mytime = []
    TTv = []
    amag_v = []   # |a| in g
    wmag_v = []   # |w| in deg/s

    flag_first = True
    flag_run = True
    start_time = time.time()

    # -[Ciclo di misura]--------------------------------------------------------
    print("t[s], ax[g], ay[g], az[g], T[C], gx[deg/s], gy[deg/s], gz[deg/s]")
    while flag_run:
        time.sleep(0.1)  # periodicità stampa

        # Verifica comunicazione (opzionale ma utile in lab)
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("ALARM: Communication lost! Check wires.")
            continue

        # Burst-read 14 byte
        sht.writeread(ACCEL_XOUT_H, 14)
        vals = sht.vals[:14]
        ax_g, ay_g, az_g, T, gx, gy, gz = parse_burst14(vals)

        # Moduli per grafico (facoltativi)
        amag = math.sqrt(ax_g*ax_g + ay_g*ay_g + az_g*az_g)
        wmag = math.sqrt(gx*gx + gy*gy + gz*gz)

        t = time.time() - start_time
        mytime.append(t)
        TTv.append(T)
        amag_v.append(amag)
        wmag_v.append(wmag)

        # Output seriale/console (requisito Task 5)
        print(f"{t:7.2f}, {ax_g: .4f}, {ay_g: .4f}, {az_g: .4f}, {T: .2f}, {gx: .3f}, {gy: .3f}, {gz: .3f}")

        # Plot
        if flag_first:
            flag_first = False
            hpT,   = ax.plot(mytime, TTv,    "o", label="T [C]")
            hpA,   = ax.plot(mytime, amag_v, "o", label="|a| [g]")
            hpW,   = ax.plot(mytime, wmag_v, "o", label="|w| [deg/s]")
            ax.grid(True)
            ax.set_xlabel("Time [s]")
            ax.set_ylabel("Value")
            ax.set_title("MPU-6050 live (ESC to exit)")
            ax.legend()
            plt.show(block=False)
            plt.tight_layout()
        else:
            hpT.set_xdata(mytime); hpT.set_ydata(TTv)
            hpA.set_xdata(mytime); hpA.set_ydata(amag_v)
            hpW.set_xdata(mytime); hpW.set_ydata(wmag_v)
            ax.relim()
            ax.autoscale_view()
            fig.canvas.draw()
            fig.canvas.flush_events()

    plt.close(fig)

finally:
    print("\nCleaning up hardware...")
    try:
        plt.close('all')
    except:
        pass
    ad2.power(False)
    ad2.close()
    print("AD2 Power Off. Process safely terminated.")


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Device: 0x68
WHO_AM_I: 0x68
t[s], ax[g], ay[g], az[g], T[C], gx[deg/s], gy[deg/s], gz[deg/s]
   0.11, -0.7393,  0.6390,  0.0514,  37.21, -168.061, -246.229,  29.298
   0.33, -0.7396,  0.6381,  0.0486,  37.21, -121.160,  240.351, -48.870
   0.76, -0.7390,  0.6394,  0.0490,  37.21, -7.817, -246.229,  48.840
   0.93, -0.7362,  0.6398,  0.0482,  37.21, -11.725, -218.870,  21.481
   1.11, -0.7382,  0.6378,  0.0496,  37.21,  27.359, -209.115, -44.962
   1.28, -0.7386,  0.6370,  0.0469,  37.21,  27.359, -226.687,  21.481
   1.45, -0.7368,  0.6425,  0.0500,  37.21,  74.260, -195.420,  1.939
   1.62, -0.7369,  0.6399,  0.0498,  37.21,  74.260,  242.321,  17.573
   1.78, -0.7386,  0.6395,  0.0469,  37.21,  105.527, -250.137,  64.473
   1.96, -0.7378,  0.6395,  0.0525,  37.21,  105.527, -242.321,  33.206
   2.13, -0.7386,  0.6395,  0.0503,  37.21,  109.435, -232.565, -37.145
   2.31, -0.7391,  0.6405,  0.0496, 

Di seguito, riportiamo le visualizzazioni separate per ogni tipo di registro di uscita (modificando opportunamente quali registri vanno in stand by tramite PWR_MGMT_2).

<img src="temperatura.png" alt="output temperatura" width="800"/>

*Figura 17: output temperatura.*

Si vede una fase di riscaldamento iniziale del sensore dovuta all'alimentazione, seguita da una stabilizzazione della temperatura intorno ai 28 °C. Nel mezzo, una salita dovuta al contatto con un dito al sensore e poi, nella discesa, alcuni picchi dovuti a soffi effettuati sul sensore. 

<img src="acceleration.png" alt="output accelerazione" width="800"/>

*Figura 18: output accelerazione.*

Dopo una prima fase in cui il sensore era posto in una posizione casuale nello spazio, si è posizionato il sensore in modo che gli assi puntassero verso l'alto, in modo da ottenere valori di accelerazione prossimi a $\pm$ 1 g sull'asse verticale e 0 g sugli altri due assi (prima z, poi y, infine x). Si notano anche alcuni picchi dovuti a movimenti improvvisi del sensore.

<img src="angular_velocity.png" alt="output giroscopio" width="800"/>

*Figura 19: output velocità angolare.*

Si nota che, una volta stabilizzato il sensore in una posizione fissa, le velocità angolari misurate sono prossime a zero, come atteso. Anche in questo caso si notano alcuni picchi dovuti a movimenti improvvisi del sensore, più accentuati su alcuni assi rispetto ad altri a seconda del piano di rotazione effettuato (prima intorno all'asse z, poi x, infine y). 

### **Task 6 – Studio statistico e esperimenti di fisica**

**Obiettivo:**
- Studiare la **precisione** e la **distribuzione** delle misure del sensore (accelerazioni, velocità angolari, temperatura), tramite uno studio statistico dei dati del sensore confrontati con le specifiche del datasheet;
- Valutare la presenza di **drift**;
- Considerare le opzioni di integrazione e datarate offerte dal dispositivo;
- Utilizzare il sensore per un esperimento di fisica; noi abbiamo scelto:
  - realizzazione di un rivelatore di tilt;
  - Studio vibrazionale dei modi normali di un modello a 2 stadi di palazzo.

**Descrizione:**
- Usando lo script del Task 5 si acquisiscono serie temporali di:
  - accelerazione (es. lungo l’asse $z$ a sensore fermo per misurare $g$),
  - velocità angolare (es. asse $z$ a sensore fermo per misurare l’offset del giroscopio),
  - temperatura nel tempo dopo l’accensione per stimare il tempo di stabilizzazione.
- Si costruiscono:
  - istogrammi delle misure (per stimare media, deviazione standard, forma della distribuzione),
  - grafici nel tempo (per osservare drift e transitori).
- Si confrontano:
  - la media di $g$ con il valore atteso (1 g) e con la **Initial Calibration Tolerance**,
  - l’offset e il rumore del giroscopio con le specifiche del datasheet,
  - il tempo di stabilizzazione della temperatura con un modello esponenziale.
- Per l’esperimento di fisica scelto - studio vibrazionale del palazzo:
  - si perturba il sistema impulsivamente in modi e punti diversi e si misurano le accelerazioni dei due stadi del modello con l'accelerometro in funzione del tempo.
  - si calcola la FFT delle acquisizioni registrate per ricavare la funzione di trasferimento del sistema (modello di palazzo a due piani): l'obiettivo è dunque quello di determinare le frequenze dei modi normali del sistema studiandole la funzione di trasferimento (caratterizzazione e individuazione delle campane di risonanza del sistema con esecuzione di fit lorentziani dove possibile e studio qualitativo altrimenti).
  - installazione di vibrodina su modulo superiore con l'obiettivo di triggerare le dinamiche dei modi normali del sistema azionandola alle frequenze naturali del sistema sopra determinate.
  - uso dell'accelerometro per misurare le vibrazioni del sistema in risonanza con successiva integrazione numerica per ricavare la legge oraria dei due moduli del modello e confrontarla con legge oraria fuori risonanza, in modo da acquisire un idea intuitiva degli effetti della risonanza sui sistemi fisici.